# LFortran: Interactive Fortran Compiler

This Demo notebook shows available magic commands in LFortran and how to use them.

# Magic Commands

Initialize some variables

In [1]:
integer :: i, j, n
n = 5
j = 0

## AST

Show the Abstract Syntax Tree (AST) after parsing (based on syntax only, no semantics):

In [2]:
%%showast
do i = 1, n
    j = j + i
end do

(TranslationUnit [(DoLoop 0 () 0 i 1 n () [(= 0 j (+ j i) ())] () ())])

## ASR

Show the Abstract Semantic Representation (ASR), which contains all the semantics.

In [3]:
%%showasr
do i = 1, n
    j = j + i
end do

(TranslationUnit (SymbolTable 1 {i: (Variable 1 i Local () () Default (Integer 4 []) Interactive Public Required .false.), j: (Variable 1 j Local () () Default (Integer 4 []) Interactive Public Required .false.), n: (Variable 1 n Local () () Default (Integer 4 []) Interactive Public Required .false.)}) [(DoLoop ((Var 1 i) (IntegerConstant 1 (Integer 4 [])) (Var 1 n) ()) [(= (Var 1 j) (IntegerBinOp (Var 1 j) Add (Var 1 i) (Integer 4 []) ()) ())])])

## LLVM

Show LLVM code:

In [4]:
%%showllvm
do i = 1, n
    j = j + i
end do

; ModuleID = 'LFortran'
source_filename = "LFortran"

@i = external global i32
@j = external global i32
@n = external global i32

define void @__lfortran_evaluate_2() {
.entry:
  store i32 0, i32* @i, align 4
  br label %loop.head

loop.head:                                        ; preds = %loop.body, %.entry
  %0 = load i32, i32* @i, align 4
  %1 = add i32 %0, 1
  %2 = load i32, i32* @n, align 4
  %3 = icmp sle i32 %1, %2
  br i1 %3, label %loop.body, label %loop.end

loop.body:                                        ; preds = %loop.head
  %4 = load i32, i32* @i, align 4
  %5 = add i32 %4, 1
  store i32 %5, i32* @i, align 4
  %6 = load i32, i32* @j, align 4
  %7 = load i32, i32* @i, align 4
  %8 = add i32 %6, %7
  store i32 %8, i32* @j, align 4
  br label %loop.head

loop.end:                                         ; preds = %loop.head
  br label %return

return:                                           ; preds = %loop.end
  ret void
}


## ASM

Show assembly code:

In [5]:
%%showasm
do i = 1, n
    j = j + i
end do

	.text
	.file	"LFortran"
	.globl	__lfortran_evaluate_3
	.p2align	4, 0x90
	.type	__lfortran_evaluate_3,@function
__lfortran_evaluate_3:
	.cfi_startproc
	movl	$0, i(%rip)
	.p2align	4, 0x90
.LBB0_1:
	movl	i(%rip), %eax
	incl	%eax
	cmpl	n(%rip), %eax
	jg	.LBB0_3
	movl	i(%rip), %eax
	incl	%eax
	movl	%eax, i(%rip)
	addl	%eax, j(%rip)
	jmp	.LBB0_1
.LBB0_3:
	retq
.Lfunc_end0:
	.size	__lfortran_evaluate_3, .Lfunc_end0-__lfortran_evaluate_3
	.cfi_endproc

	.section	".note.GNU-stack","",@progbits


## C++

Transform Fortran code to C++

In [6]:
%%showcpp
subroutine triad(a, b, scalar, c)
real, intent(in) :: a(:), b(:), scalar
real, intent(out) :: c(:)
integer :: N, i
N = size(a)
do concurrent (i = 1:N)
    c(i) = a(i) + scalar * b(i)
end do
end subroutine

#include <iostream>
#include <string>
#include <vector>
#include <cassert>
#include <cmath>
#include <complex>
#include <Kokkos_Core.hpp>
#include <lfortran_intrinsics.h>

template <typename T>
Kokkos::View<T*> from_std_vector(const std::vector<T> &v)
{
    Kokkos::View<T*> r("r", v.size());
    for (size_t i=0; i < v.size(); i++) {
        r(i) = v[i];
    }
    return r;
}


struct dimension_descriptor
{
    int32_t lower_bound, length;
};
// Forward declarations

template <typename T0, typename T1, typename T2>
void triad(T0* a, T1* b, float scalar, T2* c);

// Implementations

template <typename T0, typename T1, typename T2>
void triad(T0* a, T1* b, float scalar, T2* c)
{
    int32_t i;
    int32_t n;
    n = a->data->extent(0);
    Kokkos::parallel_for(Kokkos::RangePolicy<Kokkos::DefaultExecutionSpace>(1, n+1), KOKKOS_LAMBDA(const long i) {
        c->data->operator[](i - c->dims[0].lower_bound) = a->data->operator[](i - a->dims[0].lower_bound) + scalar*b->data->operator[](i - b->

In [7]:
%%showcpp
program doconcurrentloop_01
implicit none
real, dimension(10000) :: a, b, c
real :: scalar
integer :: i, nsize
scalar = 10
nsize = size(a)
do concurrent (i = 1:nsize)
    a(i) = 5
    b(i) = 5
end do
call triad(a, b, scalar, c)
print *, "End Stream Triad"

contains

    subroutine triad(a, b, scalar, c)
    real, intent(in) :: a(:), b(:), scalar
    real, intent(out) :: c(:)
    integer :: N, i
    N = size(a)
    do concurrent (i = 1:N)
        c(i) = a(i) + scalar * b(i)
    end do
    end subroutine

end program

#include <iostream>
#include <string>
#include <vector>
#include <cassert>
#include <cmath>
#include <complex>
#include <Kokkos_Core.hpp>
#include <lfortran_intrinsics.h>

template <typename T>
Kokkos::View<T*> from_std_vector(const std::vector<T> &v)
{
    Kokkos::View<T*> r("r", v.size());
    for (size_t i=0; i < v.size(); i++) {
        r(i) = v[i];
    }
    return r;
}


struct dimension_descriptor
{
    int32_t lower_bound, length;
};

struct f32_10000_1
{
    Kokkos::View<float*>* data;
    dimension_descriptor dims[1];
    bool is_allocated;

    f32_10000_1(Kokkos::View<float*>* data_): data{data_} {};
};

// Forward declarations
namespace {

template <typename T0, typename T1, typename T2>
void triad(T0* a, T1* b, float scalar, T2* c);
}

// Implementations
namespace {

template <typename T0, typename T1, typename T2>
void triad(T0* a, T1* b, float scalar, T2* c)
{
    int32_t i;
    int32_t n;
    n = a->data->extent(0);
    Kokkos::parallel_for(Kokkos::RangePolicy<Kokkos::

## Format Fortran code

You can use `lfortran fmt` to format Fortran code, or the `%%showfmt` command.

In [8]:
%%showfmt
subroutine triad(a, b, scalar, c); real, intent(in) :: a(:), b(:), scalar; real, intent(out) :: c(:); integer :: N, i; N = size(a); do concurrent (i = 1:N); c(i) = a(i) + scalar * b(i); end do; end subroutine

subroutine triad(a, b, scalar, c); real, intent(in) :: a(:), b(:), scalar; real, intent(out) :: c(:); integer :: N, i; N = size(a); do concurrent (i = 1:N);     c(i) = a(i) + scalar*b(i); end do; end subroutine triad
